## 1(A)

In [1]:
import numpy as np
import pandas as pd
import cvxopt
from cvxopt import solvers,matrix
from sklearn.svm import SVC
import math
from tqdm import tqdm
import multiprocessing as mp
from multiprocessing import Pool
import itertools
import threading
import pickle
from joblib import Parallel, delayed
import time
from scipy.spatial import distance
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold,cross_val_score,StratifiedKFold

In [2]:
def init():
    train = pd.read_csv("fashion_mnist/train.csv",header=None)
    val = pd.read_csv("fashion_mnist/val.csv",header=None)
    test = pd.read_csv("fashion_mnist/test.csv",header=None)
    return train,val,test

In [3]:
train,val,test = init()
c1=3
c2=4

In [4]:
def stripData(data,c1,c2):
    data = data[data[784]==c1].append(data[data[784]==c2])
    data.loc[data[784]==c1,784]=-1
    data.loc[data[784]==c2,784]=1
    Y = np.reshape(np.array(data[784]),(-1,1))
    pos = data[data[784]==1]
    neg = data[data[784]==-1]
    data = np.array(data.drop(columns=[784]))/255
    return data,Y,pos,neg

In [5]:
train = train[train[784]==c1].append(train[train[784]==c2])

In [6]:
train.loc[train[784]==c1,784]=-1
train.loc[train[784]==c2,784]=1
pos = train[train[784]==1]
neg = train[train[784]==-1]
Y = np.array(train[784])
trainD = np.array(train.drop(columns=[784]))/255

In [7]:
Y = Y.reshape((Y.shape[0],1))

In [8]:
trainD.shape

(4500, 784)

In [5]:
def createPq(data,out):
    P = (out@out.T)*(data@data.T)
    q = (-1)*np.ones((P.shape[0],1))
    return matrix(P),matrix(q)

def createGh(m):
    u = np.identity(m)
    d = (-1)*np.identity(m)
    G = np.append(u,d,axis=0)
    h = np.append(np.ones((m,1)),np.zeros((m,1)),axis=0)
    return matrix(G),matrix(h)

def createAb(out):
    m = out.shape[0]
    A = out.copy()
    A = A.T
    b = 0
    return matrix(A,(1,m),'d'),matrix(b,(1,1),'d')

In [10]:
P,q = createPq(trainD,Y)
G,h = createGh(trainD.shape[0])
A,B = createAb(Y)

In [11]:
sol=solvers.qp(P, q, G, h, A, B)

     pcost       dcost       gap    pres   dres
 0: -7.6996e+02 -9.4221e+03  5e+04  3e+00  2e-11
 1: -5.0971e+02 -5.5267e+03  1e+04  4e-01  2e-11
 2: -3.4948e+02 -1.8871e+03  3e+03  1e-01  2e-11
 3: -2.7529e+02 -9.4339e+02  1e+03  4e-02  1e-11
 4: -2.3683e+02 -5.8695e+02  6e+02  2e-02  1e-11
 5: -2.1335e+02 -3.9050e+02  3e+02  6e-03  1e-11
 6: -2.1473e+02 -2.5858e+02  6e+01  9e-04  1e-11
 7: -2.1941e+02 -2.3321e+02  2e+01  2e-04  1e-11
 8: -2.2236e+02 -2.2552e+02  3e+00  2e-05  1e-11
 9: -2.2339e+02 -2.2362e+02  2e-01  6e-07  1e-11
10: -2.2348e+02 -2.2349e+02  1e-02  3e-08  1e-11
11: -2.2348e+02 -2.2348e+02  2e-04  5e-10  1e-11
Optimal solution found.


In [12]:
alpha = np.array(sol['x'])

In [13]:
alpha.max()

0.9999999952478656

In [6]:
def stripAlpha(alpha,E):
    index = alpha < E
    alpha[index] = 0
    return alpha

In [15]:
alpha = stripAlpha(alpha,1e-4)

In [16]:
index = alpha != 0
index.sum()

494

In [7]:
def calwb(alpha,X,Y,pos,neg):
    w = X.T@(alpha*Y)
    x = (np.array(pos.drop(columns=[784]))/255)
    y = (np.array(neg.drop(columns=[784]))/255)
    b = (-1)*((x@w).min()+(y@w).max())/2
    return w,b

In [18]:
w,b = calwb(alpha,trainD,Y,pos,neg)

In [8]:
def predClasses(w,b,data):
    pred = []
    for i in data:
        temp = i.reshape((1,-1))@w+b
        if temp<0:
            pred.append(-1)
        else:
            pred.append(1)
    return pred

In [20]:
predictionTrain = predClasses(w,b,trainD)

In [9]:
def checkAccuracy(y,pred):
    y = np.array(y).reshape((-1,1))
    pred = np.array(pred).reshape((-1,1))
    check = (y==pred)
    t=0
    f=0
    for i in check:
        if i:
            t=t+1
        else:
            f=f+1
    print(t/(t+f))

In [22]:
checkAccuracy(train[784],predictionTrain)

0.9862222222222222


In [23]:
testX,testY,_,_ = stripData(test,c1,c2)
valX,valY,_,_ = stripData(val,c1,c2)

In [24]:
predictionTest = predClasses(w,b,testX)
checkAccuracy(testY,np.array(predictionTest))

0.945


In [25]:
testX.shape

(1000, 784)

In [26]:
predictionVal = predClasses(w,b,valX)
checkAccuracy(valY,predictionVal)

0.936


## 1(B)

In [10]:
def K(x,z,gamma=0.05):
    norm = np.reshape((x-z),(1,-1))
    norm = norm@norm.T
    return math.exp((-gamma)*norm)

In [11]:
def gaussianPq(data,out):
    m = data.shape[0]
    kernelMatrix=np.exp((-0.05)*np.square(distance.cdist(data,data,'euclidean')))
    P = (out@out.T)*kernelMatrix
    q = (-1)*np.ones((P.shape[0],1))
    return matrix(P),matrix(q)

In [ ]:
P,q = gaussianPq(trainD,Y)
G,h = createGh(trainD.shape[0])
A,B = createAb(Y)

In [ ]:
sol=solvers.qp(P, q, G, h, A, B)

In [ ]:
alpha = np.array(sol['x'])

In [ ]:
# 1e-2 = 98.997 96.3 96.2
# 1e-3 = 99.1 96.1 96
alpha = stripAlpha(alpha,1e-4)

In [ ]:
index = alpha != 0
index.sum()

In [12]:
def calb(alpha,X,Y,pos,neg):
    x = (np.array(pos.drop(columns=[784]))/255)
    y = (np.array(neg.drop(columns=[784]))/255)
    Y.reshape((-1,1))
    p = ((alpha*Y)*(np.exp((-0.05)*np.square(distance.cdist(X,x,'euclidean'))))).sum(axis=0).min()
    n = ((alpha*Y)*(np.exp((-0.05)*np.square(distance.cdist(X,y,'euclidean'))))).sum(axis=0).max()
    b = (-1)*(p+n)/2
    return b

In [ ]:
b = calb(alpha,trainD,Y,pos,neg)

In [ ]:
b

In [13]:
def predGaussianClasses(alpha,b,X,Y,data,return_score=False):
    pred=[]
    pred = ((alpha*Y)*(np.exp((-0.05)*np.square(distance.cdist(X,data,'euclidean'))))).sum(axis=0)
    pred = pred+b
    if(return_score):
        return pred
    index = pred < 0
    pred[index]=-1
    index = pred >= 0
    pred[index]=1
    return pred

In [ ]:
predictionTrain = predGaussianClasses(alpha,b,trainD,Y,trainD)
predictionTest = predGaussianClasses(alpha,b,trainD,Y,testX)
predictionVal = predGaussianClasses(alpha,b,trainD,Y,valX)

In [ ]:
checkAccuracy(Y,predictionTrain)
checkAccuracy(testY,predictionTest)
checkAccuracy(valY,predictionVal)

## 1(C)

In [ ]:
clfLinear = SVC(C=1.0,kernel='linear')
clfGaussian = SVC(C=1.0,kernel='rbf',gamma=0.05)

In [ ]:
clfLinear.fit(trainD, train[784])
clfGaussian.fit(trainD, train[784])

In [ ]:
print(clfLinear.support_vectors_.shape)
print(clfGaussian.support_vectors_.shape)

In [ ]:
testX,testY,_,_ = stripData(test,c1,c2)
valX,valY,_,_ = stripData(val,c1,c2)

In [ ]:
predictionTrain = clfLinear.predict(trainD)
predictionTest = clfLinear.predict(testX)
predictionVal = clfLinear.predict(valX)

In [ ]:
checkAccuracy(train[784],predictionTrain)
checkAccuracy(valY,predictionVal)
checkAccuracy(testY,predictionTest)

In [ ]:
predictionTrain = clfGaussian.predict(trainD)
predictionTest = clfGaussian.predict(testX)
predictionVal = clfGaussian.predict(valX)

In [ ]:
checkAccuracy(train[784],predictionTrain)
checkAccuracy(valY,predictionVal)
checkAccuracy(testY,predictionTest)

## 2(A)

In [14]:
globalTrain,globalVal,globalTest = init()

In [15]:
def createClf(data,classes):
    c1 = classes[0]
    c2 = classes[1]
    filteredX,filteredY,pos,neg = stripData(data,c1,c2)

    P,q = gaussianPq(filteredX,filteredY)
    G,h = createGh(filteredX.shape[0])
    A,b = createAb(filteredY)
    sol=solvers.qp(P, q, G, h, A, B)
    alpha = np.array(sol['x'])
    alpha = stripAlpha(alpha,1e-4)
    print("Alpha Done.")
    b = calb(alpha,filteredX,filteredY,pos,neg)
    print("b Done.")
    return (alpha,b,c1,c2)

In [16]:
l = list(itertools.combinations([0,1,2,3,4,5,6,7,8,9], 2))

In [ ]:
pickle_in = open("oneVone.pickle","rb")
paraList = pickle.load(pickle_in)

In [ ]:
# st = time.time()
# temp = Parallel(n_jobs=-2)(delayed(createClf)(globalTrain,i) for i in l)
# print(time.time()-st)

In [ ]:
# with open('oneVone.pickle', 'wb') as f:
#     pickle.dump(temp, f)

In [17]:
def classify(paraList,X,data):
    pred=[]
    for i in tqdm(paraList):
        filteredX,filteredY,pos,neg = stripData(X,i[2],i[3])
        pred.append(predGaussianClasses(i[0],i[1],filteredX,filteredY,data,return_score=True))
    return pred

In [ ]:
# globalTrain = np.array(globalTrain.drop(columns=[784]))/255
# globalVal = np.array(globalVal.drop(columns=[784]))/255
# globalTest = np.array(globalTest.drop(columns=[784]))/255

In [ ]:
testY = globalTest[784]
testX = np.array(globalTest.drop(columns=[784]))/255

In [ ]:
testY.shape

In [ ]:
predictions = classify(paraList,globalTrain,testX)

In [ ]:
len(predictions)

In [18]:
def findMajorityClass(predictions,l,calculations='cal'):
    pred = []
    n = len(predictions)
    m = len(predictions[0])
    for i in tqdm(range(m)):
        temp = np.zeros((10,2))
        for j in range(n):
            if(calculations=="cal"):
                if(predictions[j][i]<0):
                    temp[l[j][0],0]+=1
                    temp[l[j][0],1]+=abs(predictions[j][i])
                else:
                    temp[l[j][1],0]+=1
                    temp[l[j][1],1]+=abs(predictions[j][i])
            elif(calculations=="SVC"):
                if(predictions[j][i][0]>predictions[j][i][1]):
                    temp[l[j][0],0]+=1
                    temp[l[j][0],1]+=abs(predictions[j][i][0])
                else:
                    temp[l[j][1],0]+=1
                    temp[l[j][1],1]+=abs(predictions[j][i][1])
        m=max(temp[:,0])
        index=0
        score=-1
        for i in range(10):
            if(temp[i][0]==m and temp[i][1]>score):
                index=i
                score=temp[i][1]
        pred.append(index)
    return pred

In [ ]:
oneVonePred = findMajorityClass(predictions,l)

In [ ]:
checkAccuracy(testY,oneVonePred)

In [ ]:
valY = globalVal[784]
valX = np.array(globalVal.drop(columns=[784]))/255

In [ ]:
valX.shape

In [ ]:
predictionsVal = classify(paraList,globalTrain,valX)

In [ ]:
oneVonePredVal = findMajorityClass(predictionsVal,l)
checkAccuracy(valY,oneVonePredVal)

In [ ]:
trainY = globalTrain[784]
trainX = np.array(globalTrain.drop(columns=[784]))/255

In [ ]:
# predictionsTrain = classify(paraList,globalTrain,trainX)

In [ ]:
# oneVonePredTrain = findMajorityClass(predictionsTrain,l)
# checkAccuracy(trainY,oneVonePredTrain)

## 2(B)

In [19]:
def createSVMs(data,classes):
    c1 = classes[0]
    c2 = classes[1]
    filteredX,filteredY,pos,neg = stripData(data,c1,c2)
    filteredY.reshape((1,-1))
    clf = SVC(C=1.0,kernel='rbf',gamma=0.05,probability=True)
    clf.fit(filteredX,filteredY.ravel())
    print("Done")
    return clf

def allPredictions(clfList,data):
    pred = []
    for i in tqdm(clfList):
        x = i.predict_proba(data)
        pred.append(x)
    return pred

In [ ]:
# st = time.time()
# temp = Parallel(n_jobs=-2)(delayed(createSVMs)(globalTrain,i) for i in l)
# print(time.time()-st)

In [ ]:
# with open('oneVoneSVM.pickle', 'wb') as f:
#     pickle.dump(temp, f)

In [20]:
pickle_in = open("oneVoneSVM.pickle","rb")
clfList = pickle.load(pickle_in)

In [ ]:
testY = globalTest[784]
testX = np.array(globalTest.drop(columns=[784]))/255

In [ ]:
predictionsTest = allPredictions(clfList,testX)

In [ ]:
oneVonePredSVC = findMajorityClass(predictionsTest,l,calculations="SVC")

In [ ]:
checkAccuracy(testY,oneVonePredSVC)

In [ ]:
valY = globalVal[784]
valX = np.array(globalVal.drop(columns=[784]))/255

In [ ]:
predictionsVal = allPredictions(clfList,valX)

In [ ]:
oneVonePredVal = findMajorityClass(predictionsVal,l,calculations="SVC")
checkAccuracy(valY,oneVonePredVal)

# 2(C)

In [21]:
def confusionMatrix(test,prediction):
    mat = np.zeros((10,10))
    index=0
    for i in range(len(prediction)):
        mat[int(test[i]),int(prediction[i])] += 1
    return mat

In [ ]:
valY[12]

In [ ]:
confusion = confusionMatrix(valY,oneVonePredVal)
confusion

In [22]:
def draw(confusion):
    df_cm = pd.DataFrame(confusion.astype(int), index = [i for i in "0123456789"],
                      columns = [i for i in "0123456789"])
    plt.figure(figsize = (10,8))
    sns.set(font_scale=1.2)
    sns.heatmap(df_cm,cmap="Blues", annot=True,fmt="d",linewidth=1,annot_kws={"size": 14})
    plt.show()

# 2(D)

In [23]:
trainY = globalTrain[784]
trainX = np.array(globalTrain.drop(columns=[784]))/255

testY = globalTest[784]
testX = np.array(globalTest.drop(columns=[784]))/255

In [36]:
def findAccuracyParallel(data,c):
    clf = SVC(C=c,kernel='rbf',gamma=0.05)
    clf.fit(data[0],data[1])
    print("Done")
    
    v = clf.score(data[2],data[3])
    t = clf.score(data[4],data[5])
    return (v,t,c)
    

def verifyKfold(trainX,trainY,testX,testY):
    C = [1e-5,1e-3,1,5,10]
    accuracies = []
    skf = StratifiedKFold(n_splits=5)
    C = [1e-5,1e-3,1,5,10]
    loopTime = time.time()
    accuracies=[]
    for trainId,testId in skf.split(trainX,trainY):
        X_train, X_val = trainX[trainId], trainX[testId]
        Y_train, Y_val = trainY[trainId], trainY[testId]
        
        st = time.time()
        data = [X_train,Y_train,X_val,Y_val,testX,testY]
        
        temp = Parallel(n_jobs=-2)(delayed(findAccuracyParallel)(data,i) for i in C)
        print(time.time()-st)
        print(temp)
        accuracies.append(temp)
    print(time.time()-loopTime)
    return accuracies

In [37]:
Kfold = verifyKfold(trainX,trainY,testX,testY)

784.3307180404663
[(0.5586666666666666, 0.5726, 1e-05), (0.5586666666666666, 0.5726, 0.001), (0.878, 0.876, 1), (0.8804444444444445, 0.8784, 5), (0.8788888888888889, 0.877, 10)]
822.957213640213
[(0.5537777777777778, 0.5612, 1e-05), (0.5537777777777778, 0.5612, 0.001), (0.8768888888888889, 0.874, 1), (0.884, 0.8776, 5), (0.884, 0.8772, 10)]
856.4111366271973
[(0.5771111111111111, 0.5828, 1e-05), (0.5771111111111111, 0.5828, 0.001), (0.8848888888888888, 0.8758, 1), (0.8908888888888888, 0.8814, 5), (0.8908888888888888, 0.8802, 10)]
752.4715030193329
[(0.5762222222222222, 0.5758, 1e-05), (0.5762222222222222, 0.5758, 0.001), (0.8724444444444445, 0.873, 1), (0.8842222222222222, 0.8786, 5), (0.8842222222222222, 0.8788, 10)]
796.1141457557678
[(0.5664444444444444, 0.5686, 1e-05), (0.5664444444444444, 0.5686, 0.001), (0.8813333333333333, 0.8738, 1), (0.8824444444444445, 0.8794, 5), (0.8833333333333333, 0.8786, 10)]
4027.885692834854


In [38]:
with open('StkFold.pickle', 'wb') as f:
    pickle.dump(Kfold, f)

In [51]:
# pickle_in = open("kFold.pickle","rb")
# accList = pickle.load(pickle_in)